**Чистка таблицы**

1. Удалены дубликаты строк для всех таблиц.


2. Удалены пользователи без телефонных номеров (для таблицы, по которой будет составляться витрина). Потому что было принято соглашение, что в данной ситуации телефонный номер является индетификатором заказчика. Нужно заметить, что телефонный номер может использоваться несколькими пользователями, возможно, если они из одной семьи, поэтому имя мы не считаем идентификатором.


3. Оставлены только строки, где в столбце "новый статус" стоит "доставлено". Это один из признаков, что товар был выкуплен. Опять же, это только для таблицы выкупленных, для тотальной мы оставляем остальные строки.


4. Оставляем только строки, где в столбце "КоличествоПроданоКлиенту" стоит число >=1. Если нет, то товар не выкуплен. Для тотальной таблицы остальные строки всё равно не убираем, так как считаем потеницальные значения при выкупе.


5. Удаляем строки с доставкой для удобства будущих рассчётов, так как не считаем её отдельным товаром, полную выручку за неё рассчитаем отдельно.


6. Удаляем строки с отсутствующей ценой закупки, так как будут некорректные значения. (можно не удалять, затем они сами удалятся за счёт удаления маржи)


7. Уже не в функции удаляем чеки, где некорректная маржа. Выяснили, что таких чеков 1.43% от общего количества, для более точных данных и корректных рассчётов, сотрём их. 

**Посмотрим, как выглядит часть таблицы**

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

data = pd.read_csv('data/all_data_sas.csv', sep=";", encoding='cp1251') # open raw data file
data_saved = pd.read_csv('data/all_data_sas.csv', sep=";", encoding='cp1251')

In [2]:
data.head() #preview table

,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,СуммаДоставки,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа
0,01.05.2017 0:03,03.05.2017 0:00,4513479_TR,Доставлен,761,761,Магазины,Безналичная,Москва,КРУПНОГАБАРИТНЫЙ ТОВАР,...,49,2,1,30.04.2017 0:00,55574954-50515053535277,107117_ku13@mail.ru,Марина,ID10020734553,0,NaN
1,01.05.2017 10:18,03.05.2017 0:00,4513492_TR,К отгрузке,1 878,1 878,Pick point,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,180,1,1,30.04.2017 0:00,55574851-50484949535070,121117_yu21@yandex.ru,Гость,IDL00035982755,0,NaN
2,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,7,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00038110048,0,NaN
3,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,5,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,ID000so-4578957,0,NaN
4,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,4,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00016235553,0,NaN


**Сделаем фильтры для чистки**

In [3]:
'''
    "Clean table" function takes dataframe and the list "requests" with values 0 and 1. (1=yes, 0=no)
    Meaning of positions in the list:
    1 - delete dublicates
    2 - delete 0 telephone numbers
    3 - leave only rows with the status "delivered"
    4 - delete rows where less than 1 good was sold
    5 - delete "delivery" from types of goods in a column "Номенклатура"
    6 - delete rows where is no "цена закупки" not obligatory as automatically will be deleted
    7 - delete rows where "Маржа" is not null
    8 - delete additional information in brackets in regions
'''

def clean_table(data, requests):
    while(len(requests) < 8):
        requests.append(0)
        
    if requests[0] == 1:
        data = data[data.duplicated(keep = 'first') == False]
    
    if requests[1] == 1:
        data.drop(data.loc[data["Телефон_new"]=='0'].index, inplace=True)
        data = data[pd.notnull(data["Телефон_new"])]
    
    if requests[2] == 1:
        data.drop(data.loc[data["НовыйСтатус"]!='Доставлен'].index, inplace=True)
    
    if requests[3] == 1:
        data.drop(data.loc[data["КоличествоПроданоКлиенту"]<1].index, inplace=True)
    
    if requests[4] == 1:
        data.drop(data.loc[data["Номенклатура"]=='Доставка'].index, inplace=True)
    
    if requests[5] == 1:
        data.drop(data.loc[data["ЦенаЗакупки"]=='0'].index, inplace=True)
        
    if requests[6] == 1:
        data = data[pd.isnull(data['Маржа'])]
    
    if requests[7] == 1:
        data["Регион"] = data["Регион"].str.split('(').str[0]
    return data

**Проверим, сколько строк в таблице изначально, посмотрим, сколько их станет после удаления дубликатов**

Видим, что было 730558, стало 726866. То есть было 3692‬ повторов строк, что составляло только 0,005%. Количество уникальных чеков не изменилось, их 173958.

In [4]:
data = data_saved
requests = [1,0,0,0,0,0,0,0]
data_test = clean_table(data, requests)
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")
temp = len(data["НомерЗаказаНаСайте"].value_counts())
print(f"Всего уникальных чеков - {temp}")

Было строк в начальной таблице - 730558, стало после - 726866
Удалено 3692 строк
Удалено 0 чеков
Всего уникальных чеков - 173958


**Далее удаляем заказы, где не указан телефонный номер, смотрим на результаты.**

In [5]:
data = data_test
requests = [1,1,0,0,0,0,0,0]
data_test = clean_table(data_test, requests)
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")

Было строк в начальной таблице - 726866, стало после - 724040
Удалено 2826 строк
Удалено 798 чеков


**Оставили только заказы, у которых был статус "доставлено"**

In [6]:
data = data_test
requests = [1,1,1,0,0,0,0,0]
data_test = clean_table(data_test, requests)
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")

Было строк в начальной таблице - 724040, стало после - 293504
Удалено 430536 строк
Удалено 94464 чеков


**Удаляем заказы, где количество проданного товара < 1**

In [7]:
data = data_test
requests = [1,1,1,1,0,0,0,0]
data_test = clean_table(data_test, requests)
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")

Было строк в начальной таблице - 293504, стало после - 290657
Удалено 2847 строк
Удалено 12 чеков


**Удаляем строки с доставкой для удобства будущих рассчётов**

In [8]:
data = data_test
requests = [1,1,1,1,1,0,0,0]
data_test = clean_table(data_test, requests)
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")

Было строк в начальной таблице - 290657, стало после - 246531
Удалено 44126 строк
Удалено 6 чеков


**Удаляем строки с некорректной маржой**

In [9]:
data = data_test
requests = [1,1,1,1,1,0,0,1]
data = clean_table(data, requests)

requests = [1,0,0,0,1,0,0,1]
datatemp = clean_table(data, requests)
requests = [0,0,0,0,0,0,1,1]
datatemp = clean_table(datatemp, requests) #temporary table with incorrect transactions

series = datatemp[datatemp.columns[2]] #create series
series = series.unique() #series of incorrect transactions

data_test = data_test[~data_test["НомерЗаказаНаСайте"].isin(series)] #delete all such transactions

In [10]:
result = len(data) - len(data_test)
res_bill = len(data["НомерЗаказаНаСайте"].value_counts()) - len(data_test["НомерЗаказаНаСайте"].value_counts())
print(f"Было строк в начальной таблице - {len(data)}, стало после - {len(data_test)}")
print(f"Удалено {result} строк")
print(f"Удалено {res_bill} чеков")

Было строк в начальной таблице - 246531, стало после - 239265
Удалено 7266 строк
Удалено 970 чеков


In [11]:
data = data_test

**Смотрим чистую таблицу**

In [12]:
data

,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,СуммаДоставки,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа
0,01.05.2017 0:03,03.05.2017 0:00,4513479_TR,Доставлен,761,761,Магазины,Безналичная,Москва,КРУПНОГАБАРИТНЫЙ ТОВАР,...,49,2,1,30.04.2017 0:00,55574954-50515053535277,107117_ku13@mail.ru,Марина,ID10020734553,0,NaN
18,01.05.2017 2:09,03.05.2017 0:00,4513502_TR,Доставлен,3 299,3 234,Магазины,Безналичная,Красногорск (Красногорский район),КРУПНОГАБАРИТНЫЙ ТОВАР,...,349,1,1,01.05.2017 0:00,55574954-56545048505170,109109_mm26@gmail.com,Марина,ID59320856,0,NaN
31,01.05.2017 2:09,13.05.2017 0:00,4513529_TR,Доставлен,7 910,7 910,Магазины,Безналичная,Челябинск,КРУПНОГАБАРИТНЫЙ ТОВАР,...,0,1,1,30.04.2017 0:00,55575349-49505555535574,97117_au24@mail.ru,Анна,ID9010025983048,0,NaN
32,01.05.2017 2:09,13.05.2017 0:00,4513529_TR,Доставлен,7 910,7 910,Магазины,Безналичная,Челябинск,КРУПНОГАБАРИТНЫЙ ТОВАР,...,0,2,1,30.04.2017 0:00,55575349-49505555535574,97117_au24@mail.ru,Анна,IDL00000795856,0,NaN
36,01.05.2017 2:09,10.05.2017 0:00,4513553_TR,Доставлен,2 622,2 622,Магазины,Безналичная,Калуга,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,0,4,1,01.05.2017 0:00,55575351-51495656575170,105117_iu18@yandex.ru,Гость,IDL00000998654,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730549,30.06.2017 22:55,04.07.2017 0:00,5031477_TR,Доставлен,3 416,3 416,Магазины,Безналичная,Москва,ДЕТСКОЕ ПИТАНИЕ,...,49,7,2,30.06.2017 0:00,55574857-57485151544876,107117_ku13@mail.ru,Наталия,IDL00017315250,0,NaN
730550,30.06.2017 22:55,04.07.2017 0:00,5031477_TR,Доставлен,3 416,3 416,Магазины,Безналичная,Москва,ДЕТСКОЕ ПИТАНИЕ,...,49,8,7,30.06.2017 0:00,55574857-57485151544876,107117_ku13@mail.ru,Наталия,IDL00007784452,0,NaN
730551,30.06.2017 22:55,04.07.2017 0:00,5031477_TR,Доставлен,3 416,3 416,Магазины,Безналичная,Москва,ДЕТСКОЕ ПИТАНИЕ,...,49,6,2,30.06.2017 0:00,55574857-57485151544876,107117_ku13@mail.ru,Наталия,IDL00017315452,0,NaN
730556,30.06.2017 23:58,06.07.2017 0:00,5031783_TR,Доставлен,2 819,2 819,Магазины,Безналичная,Нижний Новгород,ДЕТСКОЕ ПИТАНИЕ,...,99,2,4,30.06.2017 0:00,55575450-53495456535575,121109_ym20@gmail.com,Мария,ID000sm-0652654,0,NaN


**Сохраняем её, если нужно**

In [13]:
data.to_csv('data_clean_market.csv', sep=";", encoding='cp1251') #save filtered dataframe